# Benchmark Results Visualization

## Text models Performance, depending on Conversation Length

In [ ]:
# importing libraries
# reading logs data


### F1

### Accuracy

### Training Time

### Inference Time

## Audio models Performance, depending on Conversation Length

### Accuracy

### Training Time

### Inference Time

### Wav2Vec Performance, depending on quantile